In [1]:
import botorch
import numpy as np
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.models import SingleTaskGP
import torch
import matplotlib.pyplot as plt
from PIL import Image
import io
import os

In [2]:
def rastigrin(x):
    "x should be a tensor of shape(batch, n)"
    d = x.shape[-1]
    return 10*d + (x**2 - 10*torch.cos(2*torch.pi*x)).sum(axis=-1)

def michalewicz(x, m=10):
    indices = torch.arange(start=1, end=1+x.shape[-1])
    return -(torch.sin(x)*torch.sin(indices * x**2 / torch.pi)**(2*m)).sum(axis=-1)

def six_hump(X):
    if len(X.shape) >= 2:
        x = X[:,0]
        y = X[:,1]
    else:
        x = X[0]
        y = X[1]
    return ((4 - 2.1*x**2 + x**4 / 3)*x**2 + x*y + (-4+4*y**2)*y**2).squeeze()

def bukin6(X):
    if len(X.shape) >= 2:
        x = X[:,0]
        y = X[:,1]
    else:
        x = X[0]
        y = X[1]
    return 100 * torch.sqrt(torch.abs(y-0.01*x**2))+ 0.01*torch.abs(x+10)

def mccormick(X):
    if len(X.shape) >= 2:
        x = X[:,0]
        y = X[:,1]
    else:
        x = X[0]
        y = X[1]
    return torch.sin(x+y) + (x-y)**2 - 1.5*x +2.5*y +1

def sum_squares(X):
    indices = torch.arange(start=1, end=1+X.shape[-1])
    return (indices * X).sum(axis=-1)

def styblinski_tang(X):
    return 0.5 * (X**4 - 16*X**2 + 5*X).sum(axis=-1)

def goldstein_price(X):
    if len(X.shape) >= 2:
        x1 = X[:,0]
        x2 = X[:,1]
    else:
        x1 = X[0]
        x2 = X[1]
    term1 = 1 + (x1 + x2 + 1)**2 * (19 - 14*x1 + 3*x1**2 - 14*x2 + 6*x1*x2 + 3*x2**2)
    term2 = 30 + (2*x1 - 3*x2)**2 * (18 - 32*x1 + 12*x1**2 + 48*x2 - 36*x1*x2 + 27*x2**2)
    return term1 * term2

In [3]:
class Function_Wrapper:
    def __init__(self, function, name, bounds, maximum):
        self.function = function
        self.name = name
        self.bounds = bounds
        self.maximum = maximum
        
Rastigrin = Function_Wrapper(rastigrin, "Rastigrin", [[-5.12,-5.12],[5.12,5.12]], 0.)
Michalewicz = Function_Wrapper(michalewicz, "Michalewicz", [[0., 0.],[torch.pi,torch.pi]], 9.66015)
Six_Hump = Function_Wrapper(six_hump, "Six_Hump", [[-3.,-2.],[3.,2.]], 1.0316)
Bukin6 = Function_Wrapper(bukin6, "Bukin6", [[-15.,-5.],[-3.,3.]], 0.)
McCormick = Function_Wrapper(mccormick, "McCormick", [[-1.5,-3.],[4.,4.]], 1.9133)
SumSquares = Function_Wrapper(sum_squares, "SumSquares", [[-10.,-10.],[10.,10.]], 0)
StyblinskiTang = Function_Wrapper(styblinski_tang, "StyblinskiTang", [[-5.,-5.],[5.,5.]], 39.16599*2)
GoldsteinPrice = Function_Wrapper(goldstein_price, "GoldsteinPrice", [[-2.,-2.],[2.,2.]], -3.)

In [4]:
def make_frame(frames, function_plot, X, Y, model, init_y, beta, i, regret, name, grid):
    #Finding max up to current iterations, to add to plot label
    max_arg = torch.argmax(init_y)
    y_val = init_y.squeeze(0)[max_arg].numpy().tolist()[0]
    x_val = model.train_inputs[0][max_arg].numpy().tolist()
    #Plotting figure first
    fig, axs = plt.subplots(1,2, figsize=(10,5), constrained_layout=True)
    ax = fig.add_subplot(121, projection='3d')
    ax.plot_surface(X, Y, function_plot, cmap="hot", alpha=0.2, label="Objective Function")
    ax.scatter(model.train_inputs[0][:-1,0], model.train_inputs[0][:-1,1], init_y[:-1].squeeze(), label="previous point")
    ax.plot_wireframe(X, Y, (model(grid).mean + np.sqrt(beta) * model(grid).stddev).detach().numpy().reshape(X.shape), alpha=0.1, color = 'g', label=r'Acquisition Function')
    ax.scatter(model.train_inputs[0][-1:,0], model.train_inputs[0][-1:,1], init_y[-1:].squeeze(), label="current points", color='r')
    ax.legend()
    #now plot regret
    axs[1].plot(np.array(regret).cumsum())
    ax.set_title(name)
    axs[1].set_title("Regret")
    fig.suptitle(f"t = {i}, min = {y_val:.4f} at {x_val[0]:.2f}, {x_val[1]:.2f}")
    #axs[1].plot(np.sqrt(np.arange(1, len(regret) + 1) * np.log(np.arange(1, len(regret) + 1))**3))
    # Save the plot to a BytesIO buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    # Convert buffer to PIL Image and store in frames
    frames.append(Image.open(buf))
    plt.close(fig)  # Close the plot to save memory
    #Preparing model for next loop

def make_graph(Wrapper: Function_Wrapper, beta_coeff, frame_num = 60):
    objective_function = lambda x: -Wrapper.function(x)
    #First, set up grid and plot function
    X = torch.linspace(Wrapper.bounds[0][0], Wrapper.bounds[1][0], 100)
    Y = torch.linspace(Wrapper.bounds[0][1], Wrapper.bounds[1][1], 100)
    X, Y = torch.meshgrid(X, Y, indexing='xy')
    grid = torch.stack([m.flatten() for m in (X, Y)], dim=-1)
    function_plot = objective_function(grid).reshape(X.shape)
    #Set up initial sampling process, x in [min, max] and then calculate corresponding y
    init_x = (Wrapper.bounds[1][0] - Wrapper.bounds[0][0]) * torch.rand((20,2), dtype=torch.float64) + Wrapper.bounds[0][0]
    init_y = objective_function(init_x).unsqueeze(-1)
    model = SingleTaskGP(train_X=init_x, train_Y=init_y)
    dims, tol = 2, 0.05
    frames = []
    regret = list()
    for i in range(1, frame_num+1):
        
        #mll = ExactMarginalLogLikelihood(likelihood=model.likelihood, model=model)
        #fit_gpytorch_mll(mll)
        #Preliminaries for optimization, initiate the acquisition function and find the max
        beta = beta_coeff * np.log(i)
        acquisition_function = botorch.acquisition.analytic.UpperConfidenceBound(model, beta, maximize=True)
    
        candidates, acquisition_value = optimize_acqf(acq_function=acquisition_function,
                                                      bounds = torch.tensor(Wrapper.bounds),
                                                      q=1,
                                                      num_restarts=10,
                                                      raw_samples=512,
                                                      options={"batch_limit": 5, "maxiter": 100})
        regret.append(Wrapper.maximum - init_y[-1].detach().item())
        #attaches a frame to frames to save at the end
        buffer = io.BytesIO()
        make_frame(frames, function_plot, X, Y, model, init_y, beta, i, regret, Wrapper.name, grid)
        #prepares model for next iteration
        next_result = (objective_function(candidates[0])).unsqueeze(-1)
        model = model.condition_on_observations(candidates.to(torch.float64), next_result.to(torch.float64))
        init_y = torch.cat((init_y, next_result.unsqueeze(-1)), dim=0)
        
    os.makedirs(os.path.dirname(f'figures/{Wrapper.name}/'), exist_ok=True)
    frames[0].save(f'figures/{Wrapper.name}/{Wrapper.name}_beta_{beta_coeff}.gif', save_all=True, append_images=frames[1:], duration=500, loop=0)

In [5]:
def test_betas(Wrapper):
    for beta in [0.1, 0.3, 1, 5, 10, 30, 50, 100]:
        make_graph(Wrapper, beta)

In [6]:
# wrappers = [SumSquares] #already tested: Michalewicz, Rastigrin, Six_Hump, Bukin6, StyblinskiTang, GoldsteinPrice, McCormick, 
# for wrapper in wrappers:
#     test_betas(wrapper)

In [7]:
test_betas(GoldsteinPrice)

/Users/josecruz/anaconda3/envs/GPR/lib/python3.11/site-packages/botorch/models/utils/assorted.py:265: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/Users/josecruz/anaconda3/envs/GPR/lib/python3.11/site-packages/botorch/models/utils/assorted.py:265: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/Users/josecruz/anaconda3/envs/GPR/lib/python3.11/site-packages/botorch/models/utils/assorted.py:265: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/Users/josecruz/anaconda3/envs/GPR/lib/python3.11/site-packages/botorch/models/utils/assorted.py:265: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
